In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install openai backoff

import openai
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import time
import backoff
import pickle


# Set up OpenAI API key
client = openai.AsyncOpenAI(
    api_key="$OPENAI_API_KEY"
)

/kaggle/input/bq-results-20241202-235745-1733184152619.csv
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 17.1 MB/s eta 0:00:00


In [2]:
data = pd.read_csv('/kaggle/input/bq-results-20241202-235745-1733184152619.csv')
data.shape

(39258, 5)

In [4]:
data = data[data['domain'] != 'time.com']
data = data[data['domain'] != 'zerohedge.com']
data = data[data['domain'] != 'breitbart.com']
data = data[data['domain'] != 'bloomberg.com']

In [5]:
data['domain'].value_counts()

domain
reuters.com           16648
theguardian.com       13274
washingtonpost.com     6827
cnn.com                1555
nytimes.com             453
forbes.com              190
theepochtimes.com       126
theblaze.com             76
infowars.com             16
Name: count, dtype: int64

In [6]:
subset_percentage = 1.0

# Find the minimum number of entries among all domains
min_entries_per_domain = data[(data['domain'] != 'infowars.com') & (data['domain'] != 'theblaze.com') & (data['domain'] != 'theepochtimes.com')]['domain'].value_counts().min()

# For using a subset of the data with balanced domains
sample_size = max(1, int(subset_percentage * len(data)))
domains = data['domain'].unique()          # Get all unique domains

# Calculate the sample size per domain
sample_size_per_domain = max(1, min(sample_size // len(domains), min_entries_per_domain))

# Perform stratified sampling
balanced_sample = data.groupby('domain', group_keys=False).apply(
    lambda x: x.sample(min(len(x), sample_size_per_domain), random_state=42)
)

data = balanced_sample.reset_index(drop=True)
data.shape

/tmp/ipykernel_30/614420470.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_sample = data.groupby('domain', group_keys=False).apply(


(1358, 5)

In [7]:
print("Bloomberg: ", data[data['domain'] == 'bloomberg.com'].shape)
print("CNN: ", data[data['domain'] == 'cnn.com'].shape)
print("Forbes: ", data[data['domain'] == 'forbes.com'].shape)
print("Infowars: ", data[data['domain'] == 'infowars.com'].shape)
print("NYTimes: ", data[data['domain'] == 'nytimes.com'].shape)
print("Reuters: ", data[data['domain'] == 'reuters.com'].shape)
print("The Epoch Times: ", data[data['domain'] == 'theepochtimes.com'].shape)
print("The Guardian: ", data[data['domain'] == 'theguardian.com'].shape)
print("Washington Post: ", data[data['domain'] == 'washingtonpost.com'].shape)
print("The Blaze: ", data[data['domain'] == 'theblaze.com'].shape)
print("Breitbart: ", data[data['domain'] == 'breitbart.com'].shape)
print("Zero Hedge: ", data[data['domain'] == 'zerohedge.com'].shape)

Bloomberg:  (0, 5)
CNN:  (190, 5)
Forbes:  (190, 5)
Infowars:  (16, 5)
NYTimes:  (190, 5)
Reuters:  (190, 5)
The Epoch Times:  (126, 5)
The Guardian:  (190, 5)
Washington Post:  (190, 5)
The Blaze:  (76, 5)
Breitbart:  (0, 5)
Zero Hedge:  (0, 5)


# Create pairs of headlines to analyze, skipping comparisons from the same source

In [8]:
import random

# Set the number of random pairs per headline
num_pairs_per_headline = 10  # Adjust this number as needed

# Initialize a list for random pairs
pairs = []

for i in range(len(data)):
    # Randomly sample indices excluding the current headline
    random_indices = random.sample(
        [j for j in range(len(data)) if j != i],
        min(num_pairs_per_headline, len(data) - 1)
    )
    
    # Create pairs with the randomly selected indices
    for j in random_indices:
        if data.iloc[i]['domain'] != data.iloc[j]['domain']:  # Skip same source
            pairs.append((
                data.iloc[i]['title'],
                data.iloc[j]['title'],
                data.iloc[i]['desc'],
                data.iloc[j]['desc'],
                data.iloc[i]['domain'],
                data.iloc[j]['domain']
            ))

# `random_pairs` now contains your randomly paired headlines

In [9]:
len(pairs)

11895

# Requesting from OpenAI

In [10]:
# Async fetch function with rate limiting
@backoff.on_exception(backoff.expo, openai.RateLimitError, max_tries=5)
async def make_api_call(prompt, session, model="gpt-4o-mini", timeout=10):
    task = asyncio.create_task(
        session.post(
            "https://api.openai.com/v1/chat/completions",
            headers={
                "Authorization": "Bearer $OPENAI_API_KEY",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}]
            },
        )
    )

    start_time = time.time()

#     while not task.done():
#         elapsed_time = time.time() - start_time
#         if elapsed_time > timeout:
#             task.cancel()
#             return {}
#         await asyncio.sleep(0.1)  # Yield control to avoid blocking the event loop

    response = await task
    return await response.json()


# Run Async Loop

In [11]:
async def process_prompts(prompts):
    connector = aiohttp.TCPConnector(limit=30000)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [make_api_call(prompt, session) for prompt in prompts]
        responses = await asyncio.gather(*tasks, return_exceptions=True)
    return responses

In [12]:
async def process_large_batch(prompts, chunk_size=5000, target_duration=0):
    results = []
    start_time = time.time()

    for i in range(0, len(prompts), chunk_size):
        chunk = prompts[i:i + chunk_size]

        # Start timing the chunk processing
        chunk_start_time = time.time()
        chunk_responses = await process_prompts(chunk)
        results.extend(chunk_responses)
        
        # Calculate elapsed time for this chunk
        chunk_duration = time.time() - chunk_start_time
        
        # Wait if chunk processing was faster than the target duration
        remaining_time = target_duration - chunk_duration
        if remaining_time > 0:
            print(f"Chunk completed in {chunk_duration:.2f}s, waiting for {remaining_time:.2f}s...")
            await asyncio.sleep(remaining_time)
        else:
            print(f"Chunk took {chunk_duration:.2f}s, starting next immediately...")

    return results

In [13]:
# Initialize an empty list for conflicts
conflicts = []

# Process pairs, generate prompts, and handle responses
prompts = [
    f"""Given the following two headlines, classify their relationship as one of the following ONLY RESPOND WITH ONE WORD (the purpose is to build a dataset for a news source credibility network via conflict detection):
    - "conflicting" if the headlines provide contradictory information.
    - "consistent" if the headlines provide complementary or similar information.
    - "unrelated" if the headlines discuss different topics.

    Headline 1: "{headline1}"
    Description 1: "{desc1}"
    Headline 2: "{headline2}"
    Description 2: "{desc2}"

    Relationship:"""
    for headline1, headline2, desc1, desc2, source1, source2 in pairs
]

responses = await process_large_batch(prompts)

# responses = await process_prompts(prompts)

# Combine responses with conflict data
for pair, response in zip(pairs, responses):
    if isinstance(response, Exception) or not isinstance(response, dict) or 'choices' not in response:
        # Skip this entry if it's an error
        continue
        
    headline1, headline2, desc1, desc2, source1, source2 = pair
    result = response['choices'][0]['message']['content'].strip().lower()

    try:
        # Normalize output for consistency
        if "conflict" in result:
            conflict_score = "conflicting"
        elif "consistent" in result:
            conflict_score = "consistent"
        elif "unrelated" in result:
            conflict_score = "unrelated"
        else:
            conflict_score = "unknown"  # Fallback for unexpected results
    except:
        continue
            

    conflicts.append({
        'source1': source1,
        'source2': source2,
        'headline1': headline1,
        'headline2': headline2,
        'conflict_score': conflict_score
    })

# Create DataFrame
conflict_df = pd.DataFrame(conflicts)

Chunk took 22.70s, starting next immediately...
Chunk took 22.21s, starting next immediately...
Chunk took 12.15s, starting next immediately...


In [14]:
responses

[{'id': 'chatcmpl-AaCl01KJ2vUciyQkr2jG8dAaQGxSe',
  'object': 'chat.completion',
  'created': 1733192210,
  'model': 'gpt-4o-mini-2024-07-18',
  'choices': [{'index': 0,
    'message': {'role': 'assistant', 'content': 'unrelated', 'refusal': None},
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 252,
   'completion_tokens': 2,
   'total_tokens': 254,
   'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
   'completion_tokens_details': {'reasoning_tokens': 0,
    'audio_tokens': 0,
    'accepted_prediction_tokens': 0,
    'rejected_prediction_tokens': 0}},
  'system_fingerprint': 'fp_3de1288069'},
 {'id': 'chatcmpl-AaCl1c5fwTP1eNW4Q60OKifKYYmok',
  'object': 'chat.completion',
  'created': 1733192211,
  'model': 'gpt-4o-mini-2024-07-18',
  'choices': [{'index': 0,
    'message': {'role': 'assistant', 'content': 'unrelated', 'refusal': None},
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 287,
   'complet

In [15]:
pd.set_option('display.max_colwidth', None)
filtered_conflict_df = conflict_df[conflict_df['conflict_score'] != 'unrelated']
filtered_conflict_df

,source1,source2,headline1,headline2,conflict_score
6,cnn.com,nytimes.com,CNN asks West Virginia residents about climate change. See their responses - CNN Video,A major new report finds some of the devastating impacts of global warming are now unavoidable.,consistent
8,cnn.com,theguardian.com,CNN asks West Virginia residents about climate change. See their responses - CNN Video,Electric cars to solar panels: tax breaks in Biden’s climate law for Americans,consistent
46,cnn.com,theguardian.com,Australia's Scott Morrison under pressure as biggest state boosts climate targets,Victorian government pledges to slash state’s carbon emissions by 50% by 2030,consistent
63,cnn.com,theguardian.com,China unveils plan to slash fossil fuels but fails to announce new emissions target,Confusion surrounds China’s energy policies as GDP and climate goals clash,conflicting
74,cnn.com,reuters.com,Manchin won't support climate or tax provisions in Democrats' economic bill,"Carney, Kerry launch global finance plan to boost climate action",conflicting
...,...,...,...,...,...
9592,washingtonpost.com,reuters.com,Three major developments in 2022 may shift the world’s climate trajectory,EU drafts plan to grow 'carbon sinks' in climate change fight,consistent
9630,washingtonpost.com,nytimes.com,Opinion | Maryland is retreating on climate action,Internal strife hampers the Greens amid climate crisis.,consistent
9634,washingtonpost.com,cnn.com,Ancient Vatican statue is latest artwork climate protesters glued hands to,Analysis: Read these 5 things about the climate crisis,consistent
9636,washingtonpost.com,nytimes.com,Ancient Vatican statue is latest artwork climate protesters glued hands to,How bad is climate change now?,consistent


In [16]:
# Save DataFrame as a CSV file
filtered_conflict_df.to_csv('conflict_scores.csv', index=False)